In [15]:
import os
import numpy as np
import gradio as gr
import pickle
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS as fa

In [2]:
import requests
import feedparser
import pandas as pd
from waybackpy import WaybackMachineCDXServerAPI
from datetime import datetime
from newspaper import Article
from datetime import date

In [3]:

os.environ["NVIDIA_API_KEY"] = input("Enter NVIDIA API KEY: ")

#Load the model via NVIDIA API
instruct_llm = ChatNVIDIA(model="meta/llama3-8b-instruct")

sys_msg0 = (
    "You are Pathwise, an AI bot that spots historical patterns/trends as it concerns improving an individuals financial situation, and notifies the user when the pattern has been spotted in the last week"
    "You are currently trying to gather the following user information: {info}"
    "Extract the information from the user input and {return_info}. If the information provided is invalid please return the word 'Invalid'"
)
sys_msg1 = (
    "You are Pathwise, an AI bot that spots historical patterns/trends as it concerns improving an individuals financial situation, and notifies the user when the pattern has been spotted in the last week"
    "You are currently trying to gather the following user information: {info}. The user has the following options to choose from: {options}. They are to provide only one number (1 to 5) in order to indicate their selection any other options are Invalid "
    "Extract the information from the user input and {return_info}. If the information provided is invalid please return the word 'Invalid'"
)
sys_msg2 = (
    "You are Pathwise, an AI bot that spots historical patterns/trends as it concerns improving an individuals financial situation, and notifies the user when the pattern has been spotted in the last week"
    "You are currently trying to gather the following user information: {info}. The user has the following options to choose from: {options}. They are to provide only one number (1 or 2) in order to indicate their selection any other options are Invalid "
    "Extract the information from the user input and {return_info}. If the information provided is invalid please return the word 'Invalid'"
)
eg1_user = {}
eg1_assist = {}
eg2_user = {}
eg2_assist = {}
eg3_user = {}
eg3_assist = {}
return_info = {}
options = {}
user_profile = {}


Enter NVIDIA API KEY:  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


In [4]:
def scrape_techcrunch_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return {
        "title": article.title,
        "authors": article.authors,
        "date": article.publish_date,
        "body": article.text
    }

In [5]:
def fetch_techcrunch_rss(url = "https://techcrunch.com/feed/"):
        """Fetch latest fintech news headlines from TechCrunch RSS feed"""
        articles = []
        try:
            feed = feedparser.parse(url)
            for entry in feed.entries:
                title = entry.get("title", "No Title")
                link = entry.get("link", "")
                articles.append((title, link))
            print(f"Fetched {len(articles)} news articles from TechCrunch")
        except Exception as e:
            print(f"Failed to fetch TechCrunch news: {e}")
        return articles

In [6]:
def get_news():
    directory = "techcrunch"
    os.makedirs(directory, exist_ok=True)
    today = date.today()
    
    articles = fetch_techcrunch_rss()
    links = [x[1] for x in articles]
    if os.path.exists(f"techcrunch/{today}-0.txt"):
        print("Already collected today's news.")
        return
    
    for i,x in enumerate(links):
        url = links[i]
        print(url)
        article = scrape_techcrunch_article(url)
        outfile = open(f"techcrunch/{today}-{i}.txt","w")
        outfile.write(article["body"])
        outfile.close()

In [7]:
def combine_current_event_with_user_profile(event_file_path, user_profile_string):
    """
    Simple concatenation of current event and user profile
    
    Args:
        event_file_path: Path to text file containing one current event
        user_profile_string: "Age: 35, Location: NYC, Education: Masters, 
                            Skills: Python Programming, Free Time: 10hrs/week, 
                            Capital: $50000, Timeline: 5 years"
    
    Returns:
        Combined text ready for embedding
    """
    # Read the current event
    with open(event_file_path, 'r') as f:
        current_event = f.read().strip()
    
    # Simple concatenation
    combined_text = f"Current Event: {current_event}\n\nUser Profile: {user_profile_string}"
    
    return combined_text

In [8]:
def process_all_current_events(current_events_folder, user_profile_string):
    """
    Process all current event files with the user profile
    
    Returns:
        List of combined texts ready for embedding
    """
    combined_texts = []
    
    for filename in os.listdir(current_events_folder):
        if filename.endswith('.txt'):
            file_path = os.path.join(current_events_folder, filename)
            
            combined_text = combine_current_event_with_user_profile(
                file_path, 
                user_profile_string
            )
            
            combined_texts.append({
                'text': combined_text,
                'source_file': filename,
                'user_profile': user_profile_string
            })
    
    return combined_texts

In [9]:
#TODO 7 - Returns a list of tuples containing the index of a historial store embedding, index of a current news embedding and a similarity score
def get_similarity_scores(current_news_embeddings, historical_store):
    """
    Compute cosine similarity between every current-news embedding
    and every historical-store embedding.

    Args:
      current_news_embeddings: Sequence of arrays, shape [(d,), …]
      historical_store:         Sequence of arrays, shape [(d,), …]

    Returns:
      List of tuples (hist_idx, curr_idx, score).
    """
    # Ensure everything is an np.ndarray
    current = [np.asarray(e, dtype=float) for e in current_news_embeddings]
    historical = [np.asarray(e, dtype=float) for e in historical_store]

    # Precompute norms
    curr_norms = [np.linalg.norm(e) for e in current]
    hist_norms = [np.linalg.norm(e) for e in historical]

    scores = []
    for curr_idx, curr_vec in enumerate(current):
        for hist_idx, hist_vec in enumerate(historical):
            # avoid division by zero
            if curr_norms[curr_idx] == 0 or hist_norms[hist_idx] == 0:
                sim = 0.0
            else:
                sim = np.dot(curr_vec, hist_vec) / (curr_norms[curr_idx] * hist_norms[hist_idx])
            scores.append((hist_idx, curr_idx, sim))

    return scores

In [10]:
#TODO 8 - Return a string with the top 3 recommend strategies 
def get_top3_strategies(financial_strategies, strategy_list):
    """
    Select the top 3 recommended strategies based on similarity scores.

    Args:
        financial_strategies: List of tuples (hist_idx, curr_idx, score)
        strategy_list:       List of strategy names (indexed by hist_idx)

    Returns:
        A string naming the top 3 strategies in descending order of score.
    """
    # Sort by score descending
    sorted_scores = sorted(financial_strategies, key=lambda x: x[2], reverse=True)

    # Pick the top 3 unique historical indices
    top_hist_idxs = []
    for hist_idx, curr_idx, score in sorted_scores:
        if hist_idx not in top_hist_idxs:
            top_hist_idxs.append(hist_idx)
        if len(top_hist_idxs) == 3:
            break

    # Map to strategy names
    top_strategies = [strategy_list[i] for i in top_hist_idxs]

    # Format the result string
    if not top_strategies:
        return "No strategies available."
    elif len(top_strategies) < 3:
        return "Top recommended strategies: " + ", ".join(top_strategies)
    else:
        return "Top 3 recommended strategies: " + ", ".join(top_strategies)

# Example usage:
# financial_strategies = [(0, 0, 0.85), (2, 0, 0.80), (1, 0, 0.78), ...]
# strategy_list = ["Mean Reversion", "Momentum", "Value Investing", ...]
# top3 = get_top3_strategies(financial_strategies, strategy_list)
# print(top3)


In [11]:
get_news()

Fetched 20 news articles from TechCrunch
https://techcrunch.com/2025/07/28/ai-data-analyst-startup-julius-nabs-10m-seed-round/
https://techcrunch.com/2025/07/28/waymo-taps-avis-to-manage-robotaxi-fleet-in-dallas/
https://techcrunch.com/2025/07/28/harmonic-the-robinhood-ceos-ai-math-startup-launches-an-ai-chatbot-app/
https://techcrunch.com/2025/07/28/flexport-sells-former-freight-unicorn-convoys-tech-two-years-after-buying-it/
https://techcrunch.com/2025/07/28/microsoft-edge-is-now-an-ai-browser-with-launch-of-copilot-mode/
https://techcrunch.com/2025/07/28/anthropic-unveils-new-rate-limits-to-curb-claude-code-power-users/
https://techcrunch.com/2025/07/28/new-york-state-cyber-chief-calls-out-trump-for-cybersecurity-cuts/
https://techcrunch.com/2025/07/28/amazon-disputes-report-that-it-raised-prices-of-popular-items-since-trump-took-office/
https://techcrunch.com/2025/07/28/why-dispos-co-founder-made-the-leap-from-social-media-to-steelmaking/
https://techcrunch.com/2025/07/28/google-ch

In [12]:
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1")

In [13]:
!tar xzvf strategy_store_idx.tgz
historical_store = fa.load_local("strategy_store_idx", embedder, allow_dangerous_deserialization=True)

strategy_store_idx/
strategy_store_idx/index.faiss
strategy_store_idx/index.pkl


In [16]:
with open('strategy_list.pkl', 'rb') as f:    # ‘rb’ = read in binary mode
    strategy_list = pickle.load(f)

In [17]:
edu_list = {}
hours_list = {}
capital_list = {}
timeframe_list = {}

In [18]:
edu_list["1"] = "High School or less"
edu_list["2"] = "Some College or Vocational Training"
edu_list["3"] = "Bachelor's Degree"
edu_list["4"] = "Master's or Professional Degree"
edu_list["5"] = "Doctorate"

In [19]:
hours_list["1"] = "1 - 3 hrs"
hours_list["2"] = "3 - 7 hrs"
hours_list["3"] = "7 - 20 hrs"
hours_list["4"] = "20 - 40 hrs"
hours_list["5"] = "40+ hrs"

In [20]:
capital_list["1"] = "< $200"
capital_list["2"] = "$200 - $500"
capital_list["3"] = "$500 - $2000"
capital_list["4"] = "$2000 - $10000"
capital_list["5"] = "$10000+"

In [21]:
timeframe_list["1"] = "less than a month"
timeframe_list["2"] = "1 - 6 months"
timeframe_list["3"] = "6 - 12 months"
timeframe_list["4"] = "1 - 3 years"
timeframe_list["5"] = "3+ years"

In [22]:
eg1_user["Age"] = "11"
eg1_assist["Age"] = "11"
eg2_user["Age"] = "eleven"
eg2_assist["Age"] = "11"
eg3_user["Age"] = "ghost"
eg3_assist["Age"] = "Invalid"
return_info["Age"] = "return only one number"


In [23]:
eg1_user["Country"] = "antigua"
eg1_assist["Country"] = "Antigua and Barbuda"
eg2_user["Country"] = "I am from Barbados"
eg2_assist["Country"] = "Barbados"
eg3_user["Country"] = "Asia"
eg3_assist["Country"] = "Invalid"
return_info["Country"] = "return a string containing one existing country"

In [24]:
eg1_user["Education"] = "3"
eg1_assist["Education"] = "3"
eg2_user["Education"] = "Two"
eg2_assist["Education"] = "2"
eg3_user["Education"] = "Pluto"
eg3_assist["Education"] = "Invalid"
options["Education"] =  "1. High School or less\n2. Some College or Vocational Training\n3. Bachelor's Degree\n4. Master's or Professional Degree\n5. Doctorate "
return_info["Education"] = "return only one number"


In [25]:
eg1_user["Skills"] = "programming"
eg1_assist["Skills"] = "Programming"
eg2_user["Skills"] = "cooking, Marketing, proGramming"
eg2_assist["Skills"] = "Cooking, Marketing, Programming"
eg3_user["Skills"] = "No"
eg3_assist["Skills"] = "Invalid"
return_info["Skills"] = "return a string of comma seperated list of skills with First Letter of each skill capitalised and lowercase for rest letters."

In [26]:
eg1_user["Hours"] = "3"
eg1_assist["Hours"] = "3"
eg2_user["Hours"] = "Two"
eg2_assist["Hours"] = "2"
eg3_user["Hours"] = "Pluto"
eg3_assist["Hours"] = "Invalid"
options["Hours"] =  "1. 1 - 3 hrs\n2. 3 - 7 hrs\n3. 7 - 20 hrs\n4. 20 - 40 hrs\n5. 40+ hrs"
return_info["Hours"] = "return only one number"

In [27]:
eg1_user["Capital"] = "3"
eg1_assist["Capital"] = "3"
eg2_user["Capital"] = "Two"
eg2_assist["Capital"] = "2"
eg3_user["Capital"] = "Pluto"
eg3_assist["Capital"] = "Invalid"
options["Capital"] =  "1. < $200\n2. $200 - $500\n3. $500 - $2000\n4. $2000 - $10000\n5. $10000+"
return_info["Capital"] = "return only one number"

In [28]:
eg1_user["TimeFrame"] = "3"
eg1_assist["TimeFrame"] = "3"
eg2_user["TimeFrame"] = "Two"
eg2_assist["TimeFrame"] = "2"
eg3_user["TimeFrame"] = "Pluto"
eg3_assist["TimeFrame"] = "Invalid"
options["TimeFrame"] =  "1. less than a month\n2. 1 - 6 months\n3. 6 - 12 months\n4. 1 - 3 years\n5. 3+ years"
return_info["TimeFrame"] = "return only one number"

In [29]:
eg1_user["Profile"] = "1"
eg1_assist["Profile"] = "1"
eg2_user["Profile"] = "Two"
eg2_assist["Profile"] = "2"
eg3_user["Profile"] = "Pluto"
eg3_assist["Profile"] = "Invalid"
options["Profile"] =  "1. yes\n2. no"
return_info["Profile"] = "return only one number"

In [30]:
prompt0 = ChatPromptTemplate.from_messages([
    ("system", sys_msg0),
    ("user", "[[{eg1_user}]]"),
    ("assistant", "{eg1_assist}"),
    ("user", "[[{eg2_user}]]"),
    ("assistant", "{eg2_assist}"),
    ("user", "[[{eg3_user}]]"),
    ("assistant", "{eg3_assist}"),
    ("user", "[[{input}]]"),
])


In [31]:
prompt1 = ChatPromptTemplate.from_messages([
    ("system", sys_msg1),
    ("user", "[[{eg1_user}]]"),
    ("assistant", "{eg1_assist}"),
    ("user", "[[{eg2_user}]]"),
    ("assistant", "{eg2_assist}"),
    ("user", "[[{eg3_user}]]"),
    ("assistant", "{eg3_assist}"),
    ("user", "[[{input}]]"),
])

In [32]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", sys_msg2),
    ("user", "[[{eg1_user}]]"),
    ("assistant", "{eg1_assist}"),
    ("user", "[[{eg2_user}]]"),
    ("assistant", "{eg2_assist}"),
    ("user", "[[{eg3_user}]]"),
    ("assistant", "{eg3_assist}"),
    ("user", "[[{input}]]"),
])

In [33]:
llm0 = prompt0 | instruct_llm | StrOutputParser()

In [34]:
llm1 = prompt0 | instruct_llm | StrOutputParser()

In [35]:
llm2 = prompt1 | instruct_llm | StrOutputParser()

In [36]:
llm3 = prompt0 | instruct_llm | StrOutputParser()

In [37]:
llm4 = prompt2 | instruct_llm | StrOutputParser()

In [38]:
print(llm0.invoke({'info':'Age','return_info':return_info["Age"],'eg1_user':eg1_user["Age"],'eg1_assist':eg1_assist["Age"],'eg2_user':eg2_user["Age"],'eg2_assist':eg2_assist["Age"],'eg3_user':eg3_user["Age"],'eg3_assist':eg3_assist["Age"],'input':"36"}))

36


In [39]:
phase = 0 #keep track of where we are in the information gathering

In [40]:
phase_rep = 0

In [41]:
def chat_with_llm(message, history):
    global phase, phase_rep
    try:
        yield {"role": "user", "content": message}

        buffer = ""
        if phase == 0:
            chat_gen = llm0.invoke({
                'info': 'Age',
                'return_info': return_info["Age"],
                'eg1_user': eg1_user["Age"],
                'eg1_assist': eg1_assist["Age"],
                'eg2_user': eg2_user["Age"],
                'eg2_assist': eg2_assist["Age"],
                'eg3_user': eg3_user["Age"],
                'eg3_assist': eg3_assist["Age"],
                'input': message
            })
            if chat_gen == 'Invalid':
                phase_rep += 1
                yield {"role": "assistant", "content": "Invalid Age. Can you enter a valid age?"}
            else:
                phase = 1
                phase_rep = 0
                user_profile["Age"] = chat_gen
                yield {"role": "assistant", "content": "Thanks. What country are you in?"}
                

        elif phase == 1:
            chat_gen = llm1.invoke({
                'info': 'Country',
                'return_info': return_info["Country"],
                'eg1_user': eg1_user["Country"],
                'eg1_assist': eg1_assist["Country"],
                'eg2_user': eg2_user["Country"],
                'eg2_assist': eg2_assist["Country"],
                'eg3_user': eg3_user["Country"],
                'eg3_assist': eg3_assist["Country"],
                'input': message
            })
            if chat_gen == 'Invalid':
                phase_rep += 1
                yield {"role": "assistant", "content": "Invalid Country. Can you enter a valid Country?"}
            else:
                phase = 2
                phase_rep = 0
                user_profile["Country"] = chat_gen
                edu_options = options['Education']
                yield {"role": "assistant", "content": f"Thanks. Can you select your highest level of education(enter item number)?\n{edu_options}"}
        elif phase == 2:
            chat_gen = llm2.invoke({
                'info': 'Education',
                'return_info': return_info["Education"],
                'eg1_user': eg1_user["Education"],
                'eg1_assist': eg1_assist["Education"],
                'eg2_user': eg2_user["Education"],
                'eg2_assist': eg2_assist["Education"],
                'eg3_user': eg3_user["Education"],
                'eg3_assist': eg3_assist["Education"],
                'options': options["Education"],
                'input': message
            })
            if chat_gen == 'Invalid':
                phase_rep += 1
                edu_options = options['Education']
                yield {"role": "assistant", "content": f"Invalid Highest Level of Education. Can you select your highest level of education(enter item number)?\n{edu_options}"}
            else:
                phase = 3
                phase_rep = 0
                user_profile["Education"] = chat_gen
                yield {"role": "assistant", "content": "Thanks. Can you enter skills that you possess separated by commas? e.g. Marketing, Programming, Writing etc."}
        elif phase == 3:
            chat_gen = llm3.invoke({
                'info': 'Skills',
                'return_info': return_info["Skills"],
                'eg1_user': eg1_user["Skills"],
                'eg1_assist': eg1_assist["Skills"],
                'eg2_user': eg2_user["Skills"],
                'eg2_assist': eg2_assist["Skills"],
                'eg3_user': eg3_user["Skills"],
                'eg3_assist': eg3_assist["Skills"],
                'input': message
            })
            if chat_gen == 'Invalid':
                phase_rep += 1
                yield {"role": "assistant", "content": "Invalid Skills. Can you enter skills that you possess separated by commas? e.g. Marketing, Programming, Writing etc."}
            else:
                phase = 4
                phase_rep = 0
                user_profile["Skills"] = chat_gen
                hours_options = options['Hours']
                yield {"role": "assistant", "content": f"Thanks. How much free time per week do you have available to dedicate to an investment(enter item number)?\n{hours_options}"}
        elif phase == 4:
            chat_gen = llm2.invoke({
                'info': 'Available Free Time',
                'return_info': return_info["Hours"],
                'eg1_user': eg1_user["Hours"],
                'eg1_assist': eg1_assist["Hours"],
                'eg2_user': eg2_user["Hours"],
                'eg2_assist': eg2_assist["Hours"],
                'eg3_user': eg3_user["Hours"],
                'eg3_assist': eg3_assist["Hours"],
                'options': options["Hours"],
                'input': message
            })
            if chat_gen == 'Invalid':
                phase_rep += 1
                hours_options = options['Hours']
                yield {"role": "assistant", "content": f"Invalid Option selected. How much free time per week do you have available to dedicate to an investment(enter item number)?\n{hours_options}"}
            else:
                phase = 5
                phase_rep = 0
                user_profile["Hours"] = chat_gen
                capital_options = options['Capital']
                yield {"role": "assistant", "content": f"Thanks. Capital available to pursue an investment(enter item number)?\n{capital_options}"}
        elif phase == 5:
            chat_gen = llm2.invoke({
                'info': 'Available Capital',
                'return_info': return_info["Capital"],
                'eg1_user': eg1_user["Capital"],
                'eg1_assist': eg1_assist["Capital"],
                'eg2_user': eg2_user["Capital"],
                'eg2_assist': eg2_assist["Capital"],
                'eg3_user': eg3_user["Capital"],
                'eg3_assist': eg3_assist["Capital"],
                'options': options["Capital"],
                'input': message
            })
            if chat_gen == 'Invalid':
                phase_rep += 1
                capital_options = options['Capital']
                yield {"role": "assistant", "content": f"Invalid Option selected. Capital available to pursue an investment(enter item number)?\n{capital_options}"}
            else:
                phase = 6
                phase_rep = 0
                user_profile["Capital"] = chat_gen
                tf_options = options["TimeFrame"]
                yield {"role": "assistant", "content": f"Thanks. Expected time frame for the pay off of the investment(enter item number)?\n{tf_options}"}        
        elif phase == 6:
            chat_gen = llm2.invoke({
                'info': 'Expected Time Frame for Return on Investment',
                'return_info': return_info["TimeFrame"],
                'eg1_user': eg1_user["TimeFrame"],
                'eg1_assist': eg1_assist["TimeFrame"],
                'eg2_user': eg2_user["TimeFrame"],
                'eg2_assist': eg2_assist["TimeFrame"],
                'eg3_user': eg3_user["TimeFrame"],
                'eg3_assist': eg3_assist["TimeFrame"],
                'options': options["TimeFrame"],
                'input': message
            })
            if chat_gen == 'Invalid':
                phase_rep += 1
                tf_options = options["TimeFrame"]
                yield {"role": "assistant", "content": f"Invalid option selected. Expected time frame for the pay off of the investment(enter item number)?\n{tf_options}"}
            else:
                phase = 7
                phase_rep = 0
                user_profile["TimeFrame"] = chat_gen
                profile_state = f"Age: {user_profile['Age']}\nCountry: {user_profile['Country']}\nEducation: {edu_list[user_profile['Education']]}\nSkills: {user_profile['Skills']}\nFree Time: {hours_list[user_profile['Hours']]}\nCapital: {capital_list[user_profile['Capital']]}\nTime Frame: {timeframe_list[user_profile['TimeFrame']]}\n"
                yield {"role": "assistant", "content": f"Thanks. This is your current profile:\n{profile_state}\nAre you satisfy with the profile? Enter 1 for yes or 2 if you desire to re-enter profile."}        
        elif phase == 7:
            chat_gen = llm4.invoke({
                'info': 'Expected Time Frame for Return on Investment',
                'return_info': return_info["Profile"],
                'eg1_user': eg1_user["Profile"],
                'eg1_assist': eg1_assist["Profile"],
                'eg2_user': eg2_user["Profile"],
                'eg2_assist': eg2_assist["Profile"],
                'eg3_user': eg3_user["Profile"],
                'eg3_assist': eg3_assist["Profile"],
                'options': options["Profile"],
                'input': message
            })
            if chat_gen == 'Invalid':
                phase_rep += 1
                profile_state = f"Age: {user_profile['Age']}\nCountry: {user_profile['Country']}\nEducation: {edu_list[user_profile['Education']]}\nSkills: {user_profile['Skills']}\nFree Time: {hours_list[user_profile['Hours']]}\nCapital: {capital_list[user_profile['Capital']]}\nTime Frame: {timeframe_list[user_profile['TimeFrame']]}\n"
                yield {"role": "assistant", "content": f"Invalid option selected. This is your current profile:\n{profile_state}\nAre you satisfy with the profile? Enter 1 for yes or 2 if you desire to re-enter profile."}
            else:
                phase = 8
                phase_rep = 0
                if chat_gen == "2":
                    phase = 0
                    yield {"role": "assistant", "content": f"Thanks. What is your age?"}
                else:
                    yield {"role": "assistant", "content": f"Thinking..."}
        elif phase == 8:
            if phase_rep == 0:
                profile_state = f"Age: {user_profile['Age']}\nCountry: {user_profile['Country']}\nEducation: {edu_list[user_profile['Education']]}\nSkills: {user_profile['Skills']}\nFree Time: {hours_list[user_profile['Hours']]}\nCapital: {capital_list[user_profile['Capital']]}\nTime Frame: {timeframe_list[user_profile['TimeFrame']]}\n"
                combined = process_all_current_events("techcrunch", profile_state)
                phase_rep += 1
                yield {"role": "assistant", "content": f"{combined[0]}"}
            elif phase_rep == 1:
                texts_to_embed = [item['text'] for item in combined]
                current_news_embeddings = embedder.embed_documents(texts_to_embed)
                #TODO 7 - Returns a list of tuples containing the index of a historial store embedding, index of a current news embedding and a similarity score
                financial_strategies = get_similarity_scores(current_news_embeddings, historical_store)
                phase_rep +=1
                yield {"role": "assistant", "content": "Strategies Assembled"}
            else:
                #TODO 8 - Return a string with the top 3 recommend strategies
                top3 = get_top3_strategies(financial_strategies,strategy_list)
                yield {"role": "assistant", "content": f"{top3}"}
                phase = 9
                phase_rep = 0
            
                    
            

    except Exception as e:
        yield {"role": "assistant", "content": f"Error: {e}"}


In [42]:
# Properly assign the initial message
initial_message = {"role": "assistant", "content": "I’m Pathwise, your AI financial advisor. I analyze historical patterns and current trends to help improve your financial situation. To begin, may I ask your age?"}

# Assign the chatbot with a list containing one message
chatbot = gr.Chatbot(
    label="Chat History",
    value=[initial_message],  
    type="messages"
)

gr.ChatInterface(
    fn=chat_with_llm,
    chatbot=chatbot,
    title="Pathwise – Personalized AI Financial Insights",
    description="Discover how historical patterns and real-time financial trends apply to you. Pathwise uses your profile to deliver tailored advice that helps you make smarter money decisions. Let's start by getting to know you.",
).queue().launch(share=True, debug=True)


/home/ubuntu/.local/lib/python3.10/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8da09f1355b62441f1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8da09f1355b62441f1.gradio.live
